### This file scrapes through moviesoundclips and gets sound clips and their transcriptions. Also it uses IBM STT to transcribe the clips

In [1]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.error import URLError, HTTPError
import wget
import os
import time
import json

from watson_developer_cloud import SpeechToTextV1

# avoid showing anaconda warnings 
import warnings
warnings.filterwarnings('ignore')

Ouptut is stored in outputJson which countains
- movie name
- speaker
- text
- filename of local sound clip
- ibm output for the transcription

In [2]:
IBM_STT = SpeechToTextV1(
        username='4ed56678-d412-4598-813b-71201ab60f90',
        password='VKhfqq2HarTo',
        x_watson_learning_opt_out=False
    )

In [3]:
os.system('rm *.wav')

0

In [4]:
fileId = 0
outputJson = []

for linkid in range(1, 298):
    link = "http://www.moviesoundclips.net/sound.php?id=" + str(linkid)
    print(link)
    try:
        o = urllib.request.urlopen(link)
    except HTTPError as e:
        # do something
        print('Error code: ', e.code)
    except URLError as e:
        # do something
        print('Reason: ', e.reason)
    else:
        r = o.read()
        soup = BeautifulSoup(r)
        
        # get movie title
        movieTitleDiv = soup.find("h2")
        if movieTitleDiv == None:
            movieTitle = "Empty"
        else:
            movieTitle = movieTitleDiv.get_text()
        print(movieTitle)
        
        # get clips
        clipDivs = soup.findAll("div", {"class": "content corner"})
        for clipDiv in clipDivs: 
            transcriptDiv = clipDiv.find("div", {"itemprop": "transcript"})
            if transcriptDiv != None:
                transcript = transcriptDiv.get_text()
                print(transcript)
                
                # restricting to clips with 
                #      one speaker 
                #      with a single sentence 
                #      no more than 10 words
                if (transcript.count('\n') == 0 and transcript.count(":") == 1 and transcript.count(" ") < 10):
                    print('   ' + str(fileId) + 'chosen transcript for download')
                    audioDivs = clipDiv.find("audio").findAll("source")
                    for audioDiv in audioDivs:
                        src = audioDiv['src']
                        
                        # downloading only wav file, mp3 and ogg also available
                        if ('wav' in src):
                            fileId = fileId + 1 
                            fname = str(fileId) + '.wav'
                            wavLink = "http://www.moviesoundclips.net" + src
                            wavFile = wget.download(wavLink, fname)
                            
                            print('   strarting ibm transcription')
                            with open(fname, 'rb') as audio_file:
                                IBM_STT_Output = IBM_STT.recognize(audio_file, content_type='audio/wav', 
                                                                   timestamps=True, word_confidence=True, 
                                                                   word_alternatives_threshold=0.001, continuous=True, 
                                                                   model='en-US_NarrowbandModel')
                            
                            entry = {}
                            entry['movie'] = movieTitle
                            entry['speaker'] = transcript[0:transcript.find(":")]
                            entry['text'] = transcript[transcript.find("\"") + 1:len(transcript) - 1]
                            entry['file'] = fname
                            entry['ibm_output'] = IBM_STT_Output
                            outputJson.append(entry)

                print('--------')
        
            

http://www.moviesoundclips.net/sound.php?id=1
Antitrust (2001)
Larry:"You guys are the greatest duo since Hewlet and Packard, Lennon and Mcarthy."
Brian:"Ben and Jerry!"
--------
Milo:"Doesn't Bill Gates have something like that?"
Gary Winston:"Bill who?"
--------
Gary Winston:"This business is binary. You're a one or a zero, alive or dead."
--------
ND 47:"I'm going to take a real shot in the dark here ok. I can offer you $42,000 a year, and a Buick if you change your mind and join us."
--------
Bob:"See! Who needs computers."
   0chosen transcript for download
   strarting ibm transcription
--------
Lisa:"Guys, we're ok. I've got it under control."
   1chosen transcript for download
   strarting ibm transcription
--------
Alice:"Milo, it wasn't all false."
Milo:"I know."
--------
Reporter:"Winston always said that any kid in a garage could put him out of business. Is that what you've done Milo?"
Milo:"Well this is a garage..."
--------
Bad Guy:"What's with the Mission Impossible 3 bu

SSLError: ("bad handshake: SysCallError(60, 'ETIMEDOUT')",)

In [5]:
outputJson
with open('output.json', 'w') as outfile:
    json.dump(outputJson, outfile)